In [196]:
import re

# Assignment 2: Analyzing large datasets with Spark. 

For this assignment, you will need to make sure you're running from a PySpark docker environment I introduced in class. You can start the docker pySpark docker environment using the following command:

```
docker run --rm -p 4040:4040 -p 8888:8888 -v $(pwd):/home/jovyan/work jupyter/all-spark-notebook
```

Make sure you run the command from the directory containing this jupyter notebook and your data folder.


</b>
# WARNING: For some reason, ipynbb document didn't always sync properly when I was pushing to github. As such, please push often and make sure your incremental changes appear on GitHub.
</b>

### Part 1

The first part will use Spark to analyze the following books, which I have downloaded for you to use from Project Gutenberg. The files are saved to the data folder.

| File name | Book Title|
|:---------:|:----------|
|43.txt | The Strange Case of Dr. Jekyll and Mr. Hyde by Robert Louis Stevenson|
|84.txt | Frankenstein; Or, The Modern Prometheus by Mary Wollstonecraft Shelley |
|398.txt  | The First Book of Adam and Eve by Rutherford Hayes Platt|
|3296.txt | The Confessions of St. Augustine by Bishop of Hippo Saint Augustine|

The objective is to explore whether we can detect similarity between books within the same topic using word-based similarity. 

The task of identifying similar texts in Natural Language Processing is crucial. A naive method for determining whether two documents are similar is to treat them as collections of words (bag of words) and use the number of words they share as a proxy for their similarity. It makes sense that two books with religion as the topic (e.g.  `398.txt` and `3296.txt`) would have more words in common than a book that discusses religion and a book that discusses science fiction (e.g. books `84.txt` and `398.txt`). 

As mentioned above, we will be using Spark to analyze the data. Although Spark is not needed for such a small example, the platform would be ideal for analyzing very large collections of documents, like those often analyzed by large corporations

This part of the assignment will rely exclusively on RDDs.

### Q1. 
Start by importing Spark and making sure your environment is set up properly for the assignment.

Import the spark context necessary to load a document as an RDD; ignore any error messages

In [152]:
### Write your code here
import pyspark
from pyspark import SparkContext
sc= SparkContext()

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at /tmp/ipykernel_36/3976109334.py:4 

### Q2 

Read in the file `43.txt` as a spark RDD and save it to a variable called `book_43`
 * make sure `book_43` is of type MapPartitionsRDD, i.e.,
   * str(type(book_43)) == "<class 'pyspark.rdd.RDD'>" should return True 

In [116]:
### Write your code here
book_43 = sc.textFile('data/43.txt')
str(type(book_43)) == "<class 'pyspark.rdd.RDD'>"

True

### Q3

How many lines does `book_43` contain?
* You can only use operations or actions on RDDs to answer the question. 
  * Code that uses methods such as `some_rdd.X().Y().Z()...` is allowed
  * Code that uses functions such as `some_func(...)` is not allowed

In [118]:
### Write your code here
book_43_rdd_size = book_43.count()
print(f"Number of objects in the RDD is {book_43_rdd_size}")

Number of objects in the RDD is 2935


### Q4 

Prior to analyzing the words contained in this book, we need to first remove the occurrences of non-alphabetical characters and numbers from the text. You can use the following function, which takes a line as input, removes digits and non-word characters, and splits it into a collection of words. 

```python
def clean_split_line(line):
    a = re.sub('\d+', '', line)
    b = re.sub('[\W]+', ' ', a)
    return b.upper().split()
```

Use the fucntion above on the variable (test_line) to see what it returns.
```python
test_line = "This is an example of that contains 234 and a dash-containing number"
```

In [119]:
### Write your code here
def clean_split_line(line):
    a = re.sub('\d+', '', line)
    b = re.sub('[\W]+', ' ', a)
    return b.upper().split()
test_line = "This is an example of that contains 234 and a dash-containing number"
clean_split_line(test_line)

['THIS',
 'IS',
 'AN',
 'EXAMPLE',
 'OF',
 'THAT',
 'CONTAINS',
 'AND',
 'A',
 'DASH',
 'CONTAINING',
 'NUMBER']

### Q5

How many words does `book_43` contain? To answer this question, you may find it useful to apply the function in a spark-fashion. 
* You can only use operations or actions on RDDs to answer the question. 
  * Code that uses methods such as `some_rdd.X().Y().Z()...` is allowed
  * Code that uses functions such as `some_func(...)` is not allowed


In [120]:
counts = book_43.flatMap(clean_split_line)
counts.take(10)
counts.count()

29116

### Q6

How many of the words in book_43 are unique? Given that words can appear in lower, upper or mixed case (ex. The, THE, the), make sure you convert the words into lower case before counting them.


In [121]:
### Write your code here
book_43_counts = book_43.flatMap(clean_split_line).map(lambda x : (x.lower()))
book_43_counts.count()
book_43_unique_counts = book_43_counts.distinct().count()
book_43_unique_counts

4296

### Q7

* Generate an `RDD` that contains the frequency of each word in `book_43`. Call the variable `book_43_counts`. Each item in the `RDD` should be a tuple with the word as the first element of the tuple and the count as the second item of the tuple. The collection should look like the following:

[('project', 88), ("the", 1807), ... ]

* Such a collection may contain a large number of words and it would be imprudent to transfer all the words onto the same machine to display it. Instead, to explore the content of such a collection, display only the first element in your list. 

* Given the random nature of this operation, the first element element displayed may be different. The first entry for me was:
```
[('project', 88)]
```

* You can only use operations or actions to answer the question. 
* Code that uses methods such as `some_rdd.X().Y().Z()...` is allowed
* Code that uses functions such as `some_func(...)` is not allowed


In [122]:
### Write your code here

word_map = book_43.flatMap(clean_split_line).map(lambda x : (x.lower(),1))
word_map.take(10)
book_43_counts = word_map.reduceByKey(lambda x,y: x+y)
book_43_counts.take(10)


[('project', 88),
 ('gutenberg', 98),
 ('ebook', 13),
 ('of', 1068),
 ('strange', 24),
 ('mr', 128),
 ('hyde', 105),
 ('robert', 3),
 ('stevenson', 3),
 ('this', 195)]

### Q8

Sort `book_43_counts` and print the 20 most frequent words in book_43. 
  * Hint: function `sortByKey` sorts a collection of tuples on the first element element of the list. You can easily change the order of the items in each element and use `sortByKey` to sort on the second item of each element in `book_43_counts`
  * You can only use operations or actions to answer the question. 
  * Code that uses methods such as `some_rdd.X().Y().Z()...` is allowed
  * Code that uses functions such as `some_func(...)` is not allowed

In [123]:
### Write your code here
#book_43_counts.sortByKey()
aa = book_43_counts.map(lambda x:(x[1],x[0])) \
.sortByKey(False)
aa.take(20)

[(1807, 'the'),
 (1068, 'of'),
 (1043, 'and'),
 (726, 'to'),
 (686, 'a'),
 (646, 'i'),
 (485, 'in'),
 (471, 'was'),
 (392, 'that'),
 (384, 'he'),
 (378, 'it'),
 (312, 'you'),
 (308, 'my'),
 (301, 'with'),
 (285, 'his'),
 (244, 'had'),
 (203, 'as'),
 (202, 'for'),
 (195, 'this'),
 (193, 'but')]

### Q9

You must have noted that the most frequent words in `book_43_counts` include stop words such as `of`, `the`, `and`, etc.

It would be inefficient to compare documents based on whether or not they contain stop words; those are common to all documents. As such, it's common to remove such stop words. The librarary `sklearn.feature_extraction` provides access to a collection of English stop words, which can be loaded using the following snippet:

```
from sklearn.feature_extraction import stop_words
stop_words.ENGLISH_STOP_WORDS
```

* Explore ENGLISH_STOP_WORDS (it's a frozen set data structure, i.e., a set that you cannot modify) by printing any 10 words from it. 
 * Hint: convert the frozen set to something you can subscript


In [124]:
### Write your code here
import sklearn
from sklearn.feature_extraction import _stop_words
stwords = _stop_words.ENGLISH_STOP_WORDS
print(list(stwords)[:10])

['had', 'herein', 'together', 'mostly', 'we', 'otherwise', 'becoming', 'same', 'again', 'ie']


### Q10

Filter out the words in `book_43_counts` by removing those that appear in the ENGLISH_STOP_WORDS.
Save the results to a new variable called `book_43_counts_filtered`
  * You can only use operarations or actions on RDDs to answer the question. 
  * Code that uses methods such as `some_rdd.X().Y().Z()...` is allowed
  * Code that uses function such as `some_func(...)` is not allowed


In [125]:
### Write your code here
list_of_stop_words = list(_stop_words.ENGLISH_STOP_WORDS)
list_of_stop_words
book_43_counts_filtered = book_43_counts.filter(lambda x: x[0] not in list_of_stop_words)

### Q11

* How many words are left in `book_43_counts_filtered` after removing the stop words

In [126]:
### Write your code here
book_43_counts_filtered.count()

4034

### Q12 

* Create a function called *process_RDD* that combines the relevant steps you proposed above to make it convenient to apply them to the remaining four books. Your function should accept an input text file path and:
 * Reads in the file as a textRDD
 * Cleans and splits the line using `clean_split_line`
 * Filters out the stop words
 * Returns a word count RDD where each item is a tuple of words and its count.
 


In [127]:
### Write your code here
def process_RDD(file):
    textRDD = sc.textFile(file)
    test_mapped = textRDD.flatMap(clean_split_line)
    text_no_stop = test_mapped.filter(lambda x: x.lower() not in list_of_stop_words)
    text_no_stop_words = text_no_stop.map(lambda x: (x, 1))
    text_count = text_no_stop_words.reduceByKey(lambda x,y: x+y)
    return text_count


### Q13 

Apply the function `process_RDD` to `book_84`, `book_398` and `book_3296` and save the results to variables `book_84_counts_filtered`, `book_398_counts_filtered` and `book_3296_counts_filtered` respectively. How many distinct words does each book contain after filtering the stop words.


In [128]:
### Write your code here
book_84_counts_filtered = process_RDD('data/84.txt').count()
book_398_counts_filtered = process_RDD('data/398.txt').count()
book_3296_counts_filtered = process_RDD('data/3296.txt').count()
print(f'{book_84_counts_filtered} is in the book_84 filtered text')
print(f'{book_398_counts_filtered} is in the book_398 filtered text')
print(f'{book_3296_counts_filtered} is in the book_3296 filtered text')

7016 is in the book_84 filtered text
2421 is in the book_398 filtered text
7293 is in the book_3296 filtered text


### Q14 

We discussed how to evaluate similarity between two texts using the number of words they share. We hypothesized that books that are similar should have more words in common than books that are dissimilar. If that holds, `book_398` and `book_3296`, which both pertain to religion, will have more words in common than, say, `book_84` and `book_398`. Test this hypothesis by writing code that compares and prints the number of words shared between `book_398` and `book_3296` and then between `book_84` and `book_398`.


In [129]:
### Write your code here
book_398_counts_filtered = process_RDD('data/398.txt')
book_3296_counts_filtered = process_RDD('data/3296.txt')
book_398_3296 = book_398_counts_filtered.join(book_3296_counts_filtered).count()
print(f'book_398 and book_3296 shared {book_398_3296} words')

book_84_counts_filtered = process_RDD('data/84.txt')
book_398_counts_filtered = process_RDD('data/398.txt')
book_84_398 = book_84_counts_filtered.join(book_398_counts_filtered).count()
print(f'book_84 and book_398 shared {book_84_398} words')


book_398 and book_3296 shared 1790 words
book_84 and book_398 shared 1691 words


### Q15

* Based on the above, do you think counting the number of shared words is a good idea as a distance metric for evaluating topic similarity? Justify your answer?
* Hint: What do *book_84* and *book_3296* have in common? 

#### Write your answer here

In [132]:
book_398_counts_filtered = process_RDD('data/84.txt')
book_3296_counts_filtered = process_RDD('data/3296.txt')
book_398_3296 = book_398_counts_filtered.join(book_3296_counts_filtered).count()
print(f'book_398 and book_3296 shared {book_398_3296} words')

book_398 and book_3296 shared 3608 words


No. Based on the common words number in the above analysis, book 398 and book 3296 have more common words than the results in book 398 and book 3296. Counting the number of shared words is NOT a good idea as a distance metric for evaluating topic similarity.

## Part II 

Another approach to estimating similarity consists of computing the Euclidean distance across a set of words. For example, suppose we have 3 documents A, B and C with the following counts for the words `evolution`, `DNA`, `biology` and `finance`. 

```python 
A = [4, 9, 6, 8]
B = [3, 7, 7, 10]
C = [15, 10, 1, 1]
```
Although all documents contain exactly the four words, the number of times these words appear in each book may be indicative of thier topic. For example, documents `A` and `B` are more likely to be business related since they contain the word `finance` more frequently (8 and 10 times respectively). Document `C` may be a technical document since it focuses on more technical words (`evolution` and `DNA`) and less on the words `finance`.

The Euclidean distance, which can be computed using the `scikit` snippet below, is more indicative of topic-relatedness between the two documents.

```python
from scipy.spatial.distance import euclidean 
print(f"The Euclidean distance between A and B is: {euclidean(A, B)}")

print(f"The Euclidean distance between A and C is: {euclidean(A, C)}")

print(f"The Euclidean distance between B and C is: {euclidean(B, C)}")
```


### Q16

To calculate the Euclidean distance, we must first identify the set of words by which we will compare the documents. Here, we will explore the words that are common to all 4 documents. We will store the data in a matrix called `counts_matrix`.

Start by finding the words that are common to all four documents after stop-word filtering and store the counts for each word in a column of `counts_matrix`. 

To take the previous example, you can generate an emtpy matrix with 3 lines (documents `A`, `B` and `C`) and 4 columns (words `evolution`, `DNA`, `biology` and `finance`) using the following code.

```python
import numpy as np
counts_matrix = np.zeros([3,4])
```

After generting the counts, you can fill the counts for a document, say `A`, using the following code:

```python
counts_matrix[0, :] = [4, 9, 6, 8] 
```
* Other than for building `counts_matrix` you should exclusively use operations or actions on the `RDD` to answer this question. 
  * Code that uses methods such as `some_rdd.X().Y().Z()...` is allowed
  * Code that uses functions such as `some_func(...)` is not allowed


In [156]:
### Write your code here
import numpy as np
shared_for_books = book_43_counts_filtered.join(book_84_counts_filtered).join(book_398_counts_filtered).join(book_3296_counts_filtered)
shared_words = shared_for_books.collect()
numbers = shared_for_books.count()
numbers
counts_matrix = np.zeros([4,1162])
A = counts_matrix[0, :] = shared_for_books.map(lambda x: x[1][0][0][0]).collect()
B = counts_matrix[1, :] = shared_for_books.map(lambda x: x[1][0][0][1]).collect()
C = counts_matrix[2, :] = shared_for_books.map(lambda x: x[1][0][1]).collect()
D = counts_matrix[3, :] = shared_for_books.map(lambda x: x[1][1]).collect()
counts_matrix


array([[98., 19., 16., ...,  1.,  1.,  1.],
       [98., 23., 21., ...,  1.,  2.,  1.],
       [93., 15., 10., ...,  1.,  1.,  1.],
       [97., 37., 16., ...,  1.,  1.,  1.]])

### Q17

Compute the Euclidean distance between `book_398` and `book_3296`, which both talk about religion and `book_84` and `book_398`. What do you conclude about using the Euclidean distance for evaluating topic relatedness across documents?


In [153]:
### Write your code here
from scipy.spatial.distance import euclidean
print(f'The Euclidean distance between book_398 and book_3296 is {euclidean(C,D)}')
print(f'The Euclidean distance between book_84 and book_398 is {euclidean(B,C)}')

The Euclidean distance between book_398 and book_3296 is 1156.6628722320086
The Euclidean distance between book_84 and book_398 is 751.6688100486809


#### Write your justification here

Euclidean distance is better than the counts of the shared words between the documnet for the similarity estimation.

### Q18

Bonus question (5 points): Can you think of a few things we could do to improve similarity between documents that pertain to the same topic. Justify your answer without giving code

#### Write your answer here


## Part III

In this part we will build some basic analytics for a dataset consisting of flight arrival and departure details for all commercial flights within the USA in one month. While this dataset can be managed using Pandas (<1M records), scaling to a yearly or longer timeframe will greatly benefit from using a distributed computing framework such as `Spark`.

Here, you should use exclusively `SparkDatFrames. 

We want to analyze this dataset to better schedule trips.  For example:
 * Avoid airlines carriers that are most often associated with delays.
 * Avoid departure days where delays are most frequent.
 * Avoid airports which are associated with delays or long taxxying time.
* etc.
 

The information about the fields contained in the data file can be found [here](https://dataverse.harvard.edu/dataset.xhtml;jsessionid=0414e25969eccd0e88ae4d64fa0b?persistentId=doi%3A10.7910%2FDVN%2FHG7NV7&version=&q=&fileTypeGroupFacet=&fileTag=%221.+Documentation%22&fileSortField=date&fileSortOrder=desc)


### Q19

Load the file `flight_info.csv` into a spark `DataFrame` called `fight_info`.

  * Note that you will need to create a sparkSession prior to loading the data
  
* How many entries does the file contain?



In [178]:
### Write your code here
from pyspark.sql import SparkSession 
pyspark = SparkSession.builder.getOrCreate()
flight_info = pyspark.read.csv("data/flight_info.csv", header=True)
flight_info.count()

450017

### Q20

Use `pySpark-SQL` or `pandas-like syntax to compute the airlines represented in this dataset
The airline information is stored in a field called UniqueCarrier
* UniqueCarrier: Represents the unique carrier code (ex.AA = American Airlines) 


In [139]:
### Write your code here
flight_info.createOrReplaceTempView('flight_info')
pyspark.sql('SELECT DISTINCT UniqueCarrier FROM flight_info').show()

+-------------+
|UniqueCarrier|
+-------------+
|           UA|
|           NK|
|           AA|
|           EV|
|           B6|
|           DL|
|           OO|
|           F9|
|           HA|
|           AS|
|           VX|
|           WN|
+-------------+



### Q21

The data file contains various other fields, two of which are useful for answering the next question.

* CRSDepTime: Represents the scheduled departure time
* DepTime: Represents the actual departure time

Compute the number of flights delayed per each carried code represented in this dataset. Sort the data by decreasing order of delays.
  * A delay is observed when `DepTime` > `CRSDepTime`


In [179]:
### Write your code here
delayed_flight = pyspark.sql('SELECT UniqueCarrier, count(*) AS Delayed_number FROM flight_info WHERE DepTime > CRSDepTime GROUP BY UniqueCarrier ORDER BY Delayed_number DESC')
delayed_flight.show()

+-------------+--------------+
|UniqueCarrier|Delayed_number|
+-------------+--------------+
|           WN|         54557|
|           DL|         28962|
|           AA|         26291|
|           UA|         19594|
|           OO|         17924|
|           EV|         12340|
|           B6|         10406|
|           AS|          4966|
|           NK|          4435|
|           F9|          3181|
|           VX|          2871|
|           HA|          2265|
+-------------+--------------+



### Q22

 Use the file `airlines.csv` to find the the complete name of the airline. Here, you are required to load the file as a pyspark DataFrame; call it `airlines_info`, and repeat the query above while including the `flights.csv `file in your query ( requires doing a `join`) so that you can also display the full name of the carrier (second column). 

The result will look (approximately) like:

```
[Row(UniqueCarrier='WN', first(_c1)='Southwest Airlines', count=SOME_count),
 Row(UniqueCarrier='DL', first(_c1)='Delta Air Lines', count=SOME_count),
 Row(UniqueCarrier='AA', first(_c1)='American Airlines', count=SOME_count),
 ...
 ]
```

The carrier code in the `airlines.csv` file is provided in the 4th (1-based) column

Note that the file `airlines.csv` does not have column header. Hence, you need to print one line of your dataset to see what names Spark gave to the columns. Use the name provided by Spark in your query.

In [180]:
### Write your code here
airlines_info = pyspark.read.csv("data/airlines.csv")
airlines_info.take(5)
airlines_info.createOrReplaceTempView('airlines_info')
delayed_flight.createOrReplaceTempView('delayed_flight')
pyspark.sql('SELECT UniqueCarrier, airlines_info._c1, Delayed_number FROM delayed_flight INNER JOIN airlines_info ON airlines_info._c3 = delayed_flight.UniqueCarrier ORDER BY Delayed_number DESC').show()            
            

+-------------+--------------------+--------------+
|UniqueCarrier|                 _c1|Delayed_number|
+-------------+--------------------+--------------+
|           WN|  Southwest Airlines|         54557|
|           DL|     Delta Air Lines|         28962|
|           AA|   American Airlines|         26291|
|           UA|     United Airlines|         19594|
|           OO|             SkyWest|         17924|
|           EV|Atlantic Southeas...|         12340|
|           B6|     JetBlue Airways|         10406|
|           AS|     Alaska Airlines|          4966|
|           NK|     Spirit Airlines|          4435|
|           F9|   Frontier Airlines|          3181|
|           VX|      Virgin America|          2871|
|           HA|   Hawaiian Airlines|          2265|
+-------------+--------------------+--------------+



### Q23

Compute the number of delays per company per day. The day is encoded as an integer in the column `DayOfWeek` in `fight_info`. You can display the day as an integer or map it into a string name of the week.
Sort the data by airline code (UniqueCarrier) and by increasing values of DayOfWeek


You results should look like the following



In [188]:
### Write your code here
delayed_flight = pyspark.sql('SELECT UniqueCarrier, DayOfWeek, count(DepTime) AS Delayed_number FROM flight_info WHERE DepTime > CRSDepTime GROUP BY UniqueCarrier, DayOfWeek ORDER BY UniqueCarrier, Delayed_number DESC').show()            

+-------------+---------+--------------+
|UniqueCarrier|DayOfWeek|Delayed_number|
+-------------+---------+--------------+
|           AA|        1|          5117|
|           AA|        7|          4645|
|           AA|        2|          3688|
|           AA|        4|          3575|
|           AA|        5|          3525|
|           AA|        3|          2941|
|           AA|        6|          2800|
|           AS|        7|           932|
|           AS|        1|           872|
|           AS|        4|           728|
|           AS|        2|           669|
|           AS|        5|           630|
|           AS|        3|           618|
|           AS|        6|           517|
|           B6|        1|          1892|
|           B6|        7|          1884|
|           B6|        2|          1787|
|           B6|        5|          1309|
|           B6|        4|          1254|
|           B6|        3|          1253|
+-------------+---------+--------------+
only showing top

### Q24  

Counting the number of delayed flights per airline is misleading, as airlines with more flights are more likley to have delays than companies with substantially fiewer flights. 

Repeat the same query above but, for each carrier, normalize the counts of delays by the total number of flights for that carrier. 


In [193]:
### Write your code here

delayed_flight = pyspark.sql('SELECT UniqueCarrier, DayOfWeek, count(DepTime) AS Delayed_number FROM flight_info WHERE DepTime > CRSDepTime GROUP BY UniqueCarrier, DayOfWeek ORDER BY UniqueCarrier, Delayed_number DESC')
flight_number = pyspark.sql('SELECT UniqueCarrier, DayOfWeek, count(*) AS flight FROM flight_info GROUP BY UniqueCarrier, DayOfWeek ORDER BY UniqueCarrier, flight DESC')
flight_number.createOrReplaceTempView('flight_number')
delayed_flight.createOrReplaceTempView('delayed_flight')
combine_flight = pyspark.sql('SELECT delayed_flight.UniqueCarrier, delayed_flight.DayOfWeek, Delayed_number, flight_number.flight FROM delayed_flight INNER JOIN flight_number ON delayed_flight.UniqueCarrier = flight_number.UniqueCarrier AND delayed_flight.DayOfWeek = flight_number.DayOfWeek ORDER BY UniqueCarrier')
combine_flight.createOrReplaceTempView('combine_flight')
pyspark.sql('SELECT UniqueCarrier,DayOfWeek,Delayed_number,flight, Delayed_number/flight as normalize FROM combine_flight ORDER BY UniqueCarrier, normalize DESC').show()

+-------------+---------+--------------+------+-------------------+
|UniqueCarrier|DayOfWeek|Delayed_number|flight|          normalize|
+-------------+---------+--------------+------+-------------------+
|           AA|        1|          5117| 12165| 0.4206329634196465|
|           AA|        7|          4645| 11678| 0.3977564651481418|
|           AA|        4|          3575|  9894|0.36133009904992924|
|           AA|        5|          3525|  9944|0.35448511665325827|
|           AA|        6|          2800|  8197| 0.3415883859948762|
|           AA|        2|          3688| 11543|0.31950099627479855|
|           AA|        3|          2941|  9711|0.30285243538255585|
|           AS|        7|           932|  2395|0.38914405010438413|
|           AS|        4|           728|  1951| 0.3731419784725782|
|           AS|        1|           872|  2454| 0.3553382233088835|
|           AS|        3|           618|  1854| 0.3333333333333333|
|           AS|        5|           630|  1960|0

### Q25 

Time the query above. How long did it take to run. 
  * Make sure you run the code a few times and compute the average run time.
  * The above should be easy to implement if you use the correct Jupyter Notebook `magic` function
  

In [150]:
### Write your code here
import numpy as np
time = (4.39, 3.79, 4.86, 4.05, 3.68)
print(f'Averge time is: {np.mean(time)} ms.')

Averge time is: 4.154 ms.


### Q26 

Use one of the techniques covered in class to accelerate this query. Time your query to see by how much the run time was improved

In [194]:
### Write your code here
flight_number.cache()
delayed_flight.cache()
combine_flight.cache()

21/11/09 08:51:41 WARN CacheManager: Asked to cache already cached data.
21/11/09 08:51:41 WARN CacheManager: Asked to cache already cached data.
21/11/09 08:51:41 WARN CacheManager: Asked to cache already cached data.


DataFrame[UniqueCarrier: string, DayOfWeek: string, Delayed_number: bigint, flight: bigint]

### Q27 

Is the departure delay (i.e., DepTime - CRSDepTime) predictive of the arrival delay (ArrTime > CRSArrTime)?
Use an approach of your choice (e.g. `skelearn` which we covered in class or `Spark`) to model as a linear regression the arrival delay as a function of the departure delay. 



In [145]:
### Write your code here
import pandas as pd
flight = pd.read_csv('data/flight_info.csv')
flight["delay_dep"] = flight["DepTime"] - flight["CRSDepTime"]
flight["delay_arr"] = flight["ArrTime"] - flight["CRSArrTime"]

print(flight.head())

flight_1 = flight[flight['delay_dep'].notna() & flight['delay_arr'].notna()]

from scipy.stats import linregress

linreg = linregress(flight_1["delay_dep"], flight_1["delay_arr"])
linreg


   Unnamed: 0  DayOfWeek UniqueCarrier  FlightNum Origin Dest  CRSDepTime  \
0           0          2            AA        494    CLT  PHX        1619   
1           1          3            AA        494    CLT  PHX        1619   
2           2          4            AA        494    CLT  PHX        1619   
3           3          5            AA        494    CLT  PHX        1619   
4           4          6            AA        494    CLT  PHX        1619   

   DepTime  TaxiOut  WheelsOff  ...  Cancelled  CancellationCode  Distance  \
0   1616.0     17.0     1633.0  ...        0.0               NaN    1773.0   
1   1614.0     13.0     1627.0  ...        0.0               NaN    1773.0   
2   1611.0     17.0     1628.0  ...        0.0               NaN    1773.0   
3   1656.0     18.0     1714.0  ...        0.0               NaN    1773.0   
4   1632.0     17.0     1649.0  ...        0.0               NaN    1773.0   

   CarrierDelay  WeatherDelay NASDelay  SecurityDelay  LateAircraftD

LinregressResult(slope=0.22335475547344275, intercept=-20.14816658160143, rvalue=0.11616972584526296, pvalue=0.0, stderr=0.002876461667436919, intercept_stderr=0.4374235372787865)